In [2]:
%cd ~/code/textrec/

/Users/kcarnold/code/textrec


In [3]:
import json
import pandas as pd
%matplotlib inline
from textrec.paths import paths

In [17]:
batch = 'gc1'

In [18]:
result_files = list(paths.gruntwork.glob(f"pairwise_mturk_{batch}*.csv"))
votes_by_HIT = pd.concat([pd.read_csv(str(f)) for f in result_files], axis=0, ignore_index=True)

In [19]:
votes_by_HIT.columns

Index(['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'Reward',
       'CreationTime', 'MaxAssignments', 'RequesterAnnotation',
       'AssignmentDurationInSeconds', 'AutoApprovalDelayInSeconds',
       'Expiration', 'NumberOfSimilarHITs', 'LifetimeInSeconds',
       'AssignmentId', 'WorkerId', 'AssignmentStatus', 'AcceptTime',
       'SubmitTime', 'AutoApprovalTime', 'ApprovalTime', 'RejectionTime',
       'RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate',
       'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'Input.stim_idx',
       'Answer.feedback', 'Answer.results', 'Approve', 'Reject'],
      dtype='object')

In [20]:
votes = []
for i, row in votes_by_HIT.iterrows():
    res = json.loads(row['Answer.results'])
    stim = row['Input.stim_idx']
    worker = row['WorkerId']
    for idx, result in enumerate(res):
        if result['isLeft'] is None:
            print("MISSING", worker, idx)
            continue
        winner = result['left']
        loser = result['right']
        if not result['isLeft']:
            winner, loser = loser, winner
        votes.append(dict(
            stim=stim,
            winning_cond=winner['condition'],
            losing_cond=loser['condition'],
            winning_text=winner['text'],
            losing_text=loser['text'],
            worker=worker,
            item_idx=idx
        ))

MISSING A39UN3PEIRSPCG 6
MISSING A39UN3PEIRSPCG 7
MISSING A39UN3PEIRSPCG 8
MISSING A39UN3PEIRSPCG 9
MISSING A39UN3PEIRSPCG 8
MISSING A39UN3PEIRSPCG 9


In [21]:
len(votes)

954

In [22]:
pd.DataFrame(votes).winning_cond.value_counts()

gated     344
norecs    337
always    273
Name: winning_cond, dtype: int64

In [23]:
from collections import Counter, defaultdict

In [24]:
def normalize_pair(winner, loser):
    if winner > loser:
        return (winner, loser, True)
    else:
        return (loser, winner, False)

In [25]:
pairs = defaultdict(Counter)
for vote in votes:
    a, b, res = normalize_pair(vote['winning_cond'], vote['losing_cond'])
    pairs[a, b][res] += 1
pairs = {f'{a} > {b}': [res[True], res[False]] for (a, b), res in pairs.items()}

In [26]:
pairs

{'norecs > gated': [183, 198],
 'norecs > always': [154, 133],
 'gated > always': [146, 140]}

In [27]:
from scipy.stats import binom_test

In [28]:
{vs: binom_test(stats) for vs, stats in pairs.items()}

{'norecs > gated': 0.473272319538945,
 'norecs > always': 0.23772336278505046,
 'gated > always': 0.7675530279201782}